In [2]:
!pip install tensorflow keras numpy tqdm requests wikipedia

  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
  Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=e7e1ceccbbbfaf7cbc802397bf216638103fb826f5cd97d377d76681753197ec
  Stored in directory: /home/simon/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import wikipedia
import concurrent.futures
from tqdm import tqdm

NUM_PAGES = 1000
MIN_LENGTH = 200
MAX_TRIES = NUM_PAGES
OUTPUT_FILE = "dataset/wikipedia.txt"
THREADS = 16

def clean_text(text):
    lines = text.splitlines()
    cleaned = [line for line in lines if not line.lstrip().startswith("==") and line.strip()]
    return "\n".join(cleaned)

def fetch_and_clean_article(_):
    try:
        title = wikipedia.random(1)
        page = wikipedia.page(title)
        text = page.content
        if len(text) >= MIN_LENGTH:
            cleaned = clean_text(text)
            return cleaned
    except Exception:
        pass
    return None

def main():
    count = 0
    buffer = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=THREADS) as executor:
        futures = [executor.submit(fetch_and_clean_article, i) for i in range(MAX_TRIES)]
        for future in tqdm(concurrent.futures.as_completed(futures), total=MAX_TRIES, desc="Downloading"):
            result = future.result()
            if result:
                buffer.append(result)
                count += 1
                if count >= NUM_PAGES:
                    break

    with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
        f.write("\n".join(buffer) + "\n")

    print(f"Downloaded {count} Wikipedia pages (attempted {MAX_TRIES} times).")

if __name__ == "__main__":
    main()

Downloading:   1%|          | 12/1000 [00:01<01:23, 11.83it/s]/home/simon/Documents/python/autocorrect-linux/venv/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/simon/Documents/python/autocorrect-linux/venv/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Downloading: 100%|██████████| 1000/1000 [01:28<00:00, 11.27it/s]

Downloaded 831 Wikipedia pages (attempted 1000 times).
